In [1]:
import kagglehub

kagglehub.login()

In [2]:
# !pip install -q -U torch immutabledict sentencepiece

In [2]:
import sys
import os
sys.path.append('/home/akobawal/gemma_pytorch/gemma')

In [3]:
from gemma_pytorch.gemma.config import GemmaConfig, get_config_for_7b
from gemma_pytorch.gemma.model import GemmaForCausalLM
import torch as pt

In [4]:
# Load the model
VARIANT = "7b" 
MACHINE_TYPE = "cuda" if pt.cuda.is_available() else "cpu"
device = pt.device(MACHINE_TYPE)
weights_dir = '/home/akobawal/LLM_Prompt_Recovery/gemma-7b-model'

model_config = get_config_for_7b()
model_config.tokenizer = "/scratch/akobawal/tokenizer.model"
model_config.quant = "quant" in VARIANT

# dtype = model_config.get_dtype()

In [5]:
print(MACHINE_TYPE)

cuda


In [6]:
pt.set_default_dtype(pt.bfloat16)
model = GemmaForCausalLM(model_config)
model.load_weights('/scratch/akobawal/gemma-7b.ckpt')
model = model.to(device).eval()

In [7]:
import pandas as pd
forum_messsages_df = pd.read_csv('/home/akobawal/LLM_Prompt_Recovery/ForumMessages.csv')
forum_messsages_df.head()

,Id,ForumTopicId,PostUserId,PostDate,ReplyToForumMessageId,Message,Medal,MedalAwardDate
0,667016,116038,2708132,11/06/2019 18:06:31,NaN,<p>Just realized that the true CEF is wrong fo...,NaN,NaN
1,667014,113221,2358604,11/06/2019 18:05:48,NaN,<p>Looks really helpful ... </p>,3.0,11/13/2019
2,667013,116036,1788308,11/06/2019 18:05:43,NaN,<p>Might someone downloaded train images 180+ ...,2.0,11/12/2019
3,667012,116035,2532029,11/06/2019 18:05:28,NaN,"<p>Nice Article, Arjit!\nJust a small point th...",NaN,NaN
4,667011,116032,413189,11/06/2019 18:02:30,666992.0,<p>Nope it was actually taking lot of space. S...,NaN,NaN


In [8]:
original_texts = forum_messsages_df['Message'][::-1]
original_texts = original_texts[5*len(original_texts)//6 + 5000:]

In [9]:
rewrite_prompts = [
    'Explain this to me like I\'m five.',
    'Convert this into a sea shanty.',
    'Make this rhyme.',
    'write in style of a Novel',
    'write in style of a Short story',
    'write in style of a Poetry',
    'write in style of a Drama (play)',
    'write in style of a Essay',
    'write in style of a Biography',
    'write in style of a Autobiography',
    'write in style of a Memoir',
    'write in style of a Satire',
    'write in style of a Fable',
    'write in style of a Allegory',
    'write in style of a Parable',
    'write in style of a Epic',
    'write in style of a Lyric poetry',
    'write in style of a Sonnet',
    'write in style of a Haiku',
    'write in style of a Ballad',
    'write in style of a Villanelle',
    'write in style of a Ode',
    'write in style of a Elegy',
    'write in style of a Free verse',
    'write in style of a Prose poetry',
    'write in style of a Flash fiction',
    'write in style of a Science fiction',
    'write in style of a Fantasy',
    'write in style of a Mystery',
    'write in style of a Thriller',
    'write in style of a Romance',
    'write in style of a Historical fiction',
    'write in style of a Gothic fiction',
    'write in style of a Haiku',
    'write in style of a Light Poetry',
    'write in style of a verse',
    'write in style of a Elegy',
    'write in style of a Sonnet',
    'write in style of a Limerick',
]

In [10]:
import random
random.seed(0)

USER_CHAT_TEMPLATE = "<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"

rewrite_data = []
x = 0 

In [ ]:
for original_text in original_texts:
    rewrite_prompt = random.choice(rewrite_prompts)
    prompt = f'{rewrite_prompt}\n{original_text}'
    if len(prompt)>8162: continue
    rewritten_text = model.generate(
        USER_CHAT_TEMPLATE.format(prompt=prompt),
        device=device,
        output_len=100,
    )
    rewrite_data.append({
        'original_text': original_text,
        'rewrite_prompt': rewrite_prompt,
        'rewritten_text': rewritten_text,
    })
    x+=1
    if x%100==0:
        rewrite_data_df = pd.DataFrame(rewrite_data)
        rewrite_data_df.to_csv('gen_data.csv', sep=',', mode='a', index=False, encoding='utf-8')
        rewrite_data = []
        if x%1000==0: print(x)

1000
2000


In [14]:
print(rewrite_data[1]['original_text'] + rewrite_data[1]['rewrite_prompt'] + '\n' + rewrite_data[1]['rewritten_text'])

<p>Looks really helpful ... </p>write in style of a Science fiction
<p>its all of it </p>﻿
<p>My best friend and a role model</p>﻿
<p>"Your a robot</p><em>and</em><p>You can't read!"</p>
<p>I'm going to start writing and making videos because you gave me the courage to do that</p>
Your best friend and a role model
<p>They are great to you</p>
<p


In [17]:
import time

start_time = time.time()

for original_text in original_texts:
    rewrite_prompt = random.choice(rewrite_prompts)
    prompt = f'{rewrite_prompt}\n{original_text}'
    rewritten_text = model.generate(
        USER_CHAT_TEMPLATE.format(prompt=prompt),
        device=device,
        output_len=100,
    )
    rewrite_data.append({
        'original_text': original_text,
        'rewrite_prompt': rewrite_prompt,
        'rewritten_text': rewritten_text,
    })

end_time = time.time()

# Calculate execution time
execution_time = end_time - start_time

In [21]:
print(execution_time)

242.08665442466736
